In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
import joblib
import os
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import layers
!pip install keras_tuner
import keras_tuner as kt



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.7 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Open master df

In [7]:
# Open master
df_train= pd.read_feather('/content/drive/MyDrive/Adv MLA/AT3_Flight/train_data.feather')
df_test= pd.read_feather('/content/drive/MyDrive/Adv MLA/AT3_Flight/test_data.feather')


In [8]:
# Calculate average price per route
route_avg_price = (
    df_train.groupby(['startingAirport', 'destinationAirport'])['totalFare']
    .mean()
    .reset_index()
    .rename(columns={'totalFare': 'average_price'})
)


# Merge this back to the original dataset
df_train = df_train.merge(route_avg_price, on=['startingAirport', 'destinationAirport'], how='left')
df_test = df_test.merge(route_avg_price, on=['startingAirport', 'destinationAirport'], how='left')


In [9]:
# Calculate average distance to the dataset
route_avg_distance = (
    df_train.groupby(['startingAirport', 'destinationAirport'])['totalTravelDistance']
    .mean()
    .reset_index()
    .rename(columns={'totalTravelDistance': 'average_distance'})
)

df_train = df_train.merge(route_avg_distance, on=['startingAirport', 'destinationAirport'], how='left')
df_test = df_test.merge(route_avg_distance, on=['startingAirport', 'destinationAirport'], how='left')


In [10]:
# Calculate unique routes in the dataset
# Create a new column to represent the unique route, combining airports alphabetically, store as string
def unique_route(df):
    df['route'] = df[['startingAirport', 'destinationAirport']].apply(
        lambda x: str(tuple(sorted(x))), axis=1
    )
    return df

df_train = unique_route(df_train)
df_test = unique_route(df_test)

# Modelling

Steps
1. Encode the categories
2. Normalise
3. Split and train
4. train
5. Eval

### Preprocess data

In [34]:

# Preprocess the features
# Define the features and target
def preprocess_drop(df):
    df = df.drop([ 'searchDate', 'flightDate','segmentsArrivalAirportCode'], axis=1)

    return df

# #Label encode for airports
# def process_airports(df):
#     le = LabelEncoder()
#     all_airports = sorted(set(df['startingAirport']).union(df['destinationAirport']))
#     le.fit(all_airports)

#     df['startingAirport'] = le.transform(df['startingAirport'])
#     df['destinationAirport'] = le.transform(df['destinationAirport'])
#     # print dictionary of the label encoder for airports with original values and the encoded values
#     print(dict(zip(le.classes_, le.transform(le.classes_))))

#     return df, le

# def process_airports_test(df):

#     df['startingAirport'] = le.transform(df['startingAirport'])
#     df['destinationAirport'] = le.transform(df['destinationAirport'])
#     return df



#Features to process
boolean_cols = ['isNonStop']
ohe_cols = ['AirlineNameScore', 'CabinCode']
scale_cols = ['DepartureTimeHour','date_diff_days', 'CabinCode','average_distance', 'average_price']
scale_cols = list(set(scale_cols) - set(ohe_cols))

# Encode the boolean column
def process_boolean(df):
    df[boolean_cols] = df[boolean_cols].astype(int)
    return df

# scale data
def process_scale(df):
    scaler = StandardScaler()
    df[scale_cols] = scaler.fit_transform(df[scale_cols])
    return df, scaler

def process_scale_test(df):
    df[scale_cols] = scaler.transform(df[scale_cols])
    return df

# weekday to get cos and sine
def process_weekday(df):
    df['weekday_sin'] = np.sin(2 * np.pi * df['weekday'] / 7)
    df['weekday_cos'] = np.cos(2 * np.pi * df['weekday'] / 7)
    df.drop('weekday', axis=1, inplace=True)
    return df

# onehot encode cabin code
def process_ohe(df):
    print("Columns before one-hot encoding:", df.columns)
    print("Columns to one-hot encode:", ohe_cols)
    df = pd.get_dummies(df, columns=ohe_cols)
    return df

# Preprocess the features
def preprocess_features(df):
    # df, le = process_airports(df)
    # print the columns in df
    print('processairport',df.columns)
    df = process_boolean(df)
    print('processboolean',df.columns)
    df = process_weekday(df)
    print('processweekday',df.columns)
    df = process_ohe(df)
    print('processohe',df.columns)
    df, scaler = process_scale(df)
    print('processscale',df.columns)

    return df, le, scaler

def preprocess_features_test(df):
    # df = process_airports_test(df)
    df = process_boolean(df)
    df = process_weekday(df)
    df = process_ohe(df)
    df = process_scale_test(df)
    return df

def prepar_data_set(data_df):
    categoy_features = ['startingAirport', 'destinationAirport', 'AirlineNameScore', 'CabinCode', 'route', 'weekday']
    numerique_features = ['DepartureTimeHour', 'date_diff_days', 'average_distance', 'average_price']
    encoders = {}
    for col in categoy_features:
        encoder = LabelEncoder()
        data_df[col] = encoder.fit_transform(data_df[col])
        encoders[col] = encoder
    return data_df,categoy_features,numerique_features, encoders

def prepare_test_set(data_df):
    categoy_features = ['startingAirport', 'destinationAirport', 'AirlineNameScore', 'CabinCode', 'route',  'weekday']

    for col in categoy_features:
        print(col)
        data_df[col] = encoders[col].transform(data_df[col])
    return data_df



In [12]:
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=42)

In [13]:
train = preprocess_drop(df_train.copy())
val = preprocess_drop(df_val.copy())
test = preprocess_drop(df_test.copy())

In [35]:
#trainset
data_df,categoy_features,numerique_features, encoders = prepar_data_set(train)
scaler = StandardScaler()
data_df[numerique_features] = scaler.fit_transform(data_df[numerique_features])

data_df_test = prepare_test_set(test)
data_df_test[numerique_features] = scaler.transform(data_df_test[numerique_features])


startingAirport
destinationAirport
AirlineNameScore
CabinCode
route
weekday


In [18]:
# Define your distribution strategy for TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.


Running on TPU  


In [19]:
# Create tensorflow nn


def create_nn(hp):
    # Define the input layers
    input_layers = []
    output_layers = []
    for col in categoy_features:
        input_layer = Input(shape=(1,), name=col)
        embedding = Embedding(input_dim=int(data_df[col].max()) + 1, output_dim=20)(input_layer)
        embedding = Flatten()(embedding)
        input_layers.append(input_layer)
        output_layers.append(embedding)
    for col in numerique_features:
        input_layer = Input(shape=(1,), name=col)
        input_layers.append(input_layer)
        output_layers.append(input_layer)
    # Concatenate the layers
    x = Concatenate()(output_layers)

    # Tune the number of layers and units
    for i in range(hp.Int('num_layers', 1, 3)):  # 1 to 3 hidden layers
        x = Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),
                  activation=hp.Choice('activation_' + str(i), ['relu', 'tanh']))(x)

    x = Dense(1)(x)

    model = Model(inputs=input_layers, outputs=x)

    # Tune the learning rate
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='mean_squared_error')
    return model

input_dict={
    'startingAirport': data_df['startingAirport'],
    'destinationAirport': data_df['destinationAirport'],
    'AirlineNameScore': data_df['AirlineNameScore'],
    'CabinCode': data_df['CabinCode'],
    'DepartureTimeHour': data_df['DepartureTimeHour'],
    'date_diff_days': data_df['date_diff_days'],
    'average_distance': data_df['average_distance'],
    'average_price': data_df['average_price'],
    'weekday': data_df['weekday'],
    'route': data_df['route']
}

# model = create_nn()
# history = model.fit(input_dict, data_df['totalFare'], epochs=50, batch_size=8192, validation_split=0.2)

# Evaluate the model
# predict
input_dict_test={
    'startingAirport': data_df_test['startingAirport'],
    'destinationAirport': data_df_test['destinationAirport'],
    'AirlineNameScore': data_df_test['AirlineNameScore'],
    'CabinCode': data_df_test['CabinCode'],
    'DepartureTimeHour': data_df_test['DepartureTimeHour'],
    'date_diff_days': data_df_test['date_diff_days'],
    'average_distance': data_df_test['average_distance'],
    'average_price': data_df_test['average_price'],
    'weekday': data_df_test['weekday'],
    'route': data_df_test['route']
}
# Define the Keras Tuner
tuner = kt.Hyperband(create_nn,
                     objective='val_loss',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')
# Run the search
print('batchsize: ', 32 * strategy.num_replicas_in_sync)
tuner.search(input_dict,
             data_df['totalFare'],
             epochs=10,
             batch_size=32 * strategy.num_replicas_in_sync,
             validation_split=0.2)

# Get the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)


y_pred = model.predict(input_dict_test)
mae = mean_absolute_error(data_df_test['totalFare'], y_pred)
rmse = root_mean_squared_error(data_df_test['totalFare'], y_pred)
print(f'Mean Absolute Error: {mae}')
print(f'Root Mean Squared Error: {rmse}')

# # Save the model
# model.save('../models/nn/nn_model')





Trial 30 Complete [00h 07m 31s]
val_loss: 13147.15625

Best val_loss So Far: 12242.58203125
Total elapsed time: 01h 44m 13s
39896/39896 [==============================] - 69s 2ms/step
Mean Absolute Error: 324.3405456542969
Root Mean Squared Error: 376.44879150390625


In [21]:
# Save the model to Google Drive
model_path = '/content/drive/My Drive/Adv MLA/AT3_Flight/best_model.keras'
model.save(model_path)

In [4]:
#open saved model
from tensorflow.keras.models import load_model
reconstructed_model = load_model("/content/drive/My Drive/Adv MLA/AT3_Flight/best_model.keras")

TypeError: Could not deserialize class 'Functional' because its parent module keras.src.engine.functional cannot be imported. Full object config: {'module': 'keras.src.engine.functional', 'class_name': 'Functional', 'config': {'name': 'model_1', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_input_shape': [None, 1], 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'startingAirport'}, 'registered_name': None, 'name': 'startingAirport', 'inbound_nodes': []}, {'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_input_shape': [None, 1], 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'destinationAirport'}, 'registered_name': None, 'name': 'destinationAirport', 'inbound_nodes': []}, {'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_input_shape': [None, 1], 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'AirlineNameScore'}, 'registered_name': None, 'nam

In [5]:
# open /content/drive/MyDrive/Adv MLA/AT3_Flight/best_model.h5
import tensorflow as tf
reconstructed_model = tf.keras.models.load_model('/content/drive/MyDrive/Adv MLA/AT3_Flight/best_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
# summarise model
reconstructed_model.summary()

# parameters for model


Model: "model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ startingAirport           │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ destinationAirport        │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ AirlineNameScore          │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ CabinCode (InputLayer)    │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ route (InputLayer)        │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ weekday (InputLayer)      │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_6 (Embedding)   │ (None, 1, 20)          │            260 │ startingAirport[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_7 (Embedding)   │ (None, 1, 20)          │            320 │ destinationAirport[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_8 (Embedding)   │ (None, 1, 20)          │             80 │ AirlineNameScore[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_9 (Embedding)   │ (None, 1, 20)          │            140 │ CabinCode[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_10 (Embedding)  │ (None, 1, 20)          │          2,300 │ route[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_11 (Embedding)  │ (None, 1, 20)          │            140 │ weekday[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_6 (Flatten)       │ (None, 20)             │              0 │ embedding_6[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_7 (Flatten)       │ (None, 20)             │              0 │ embedding_7[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_8 (Flatten)       │ (None, 20)             │              0 │ embedding_8[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_9 (Flatten)       │ (None, 20)             │              0 │ embedding_9[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_10 (Flatten)      │ (None, 20)             │              0 │ embedding_10[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_11 (Flatten)      │ (None, 20)             │              0 │ embedding_11[0][0]     │
├──────────────────────

 Total params: 138,507 (541.05 KB)

 Trainable params: 138,505 (541.04 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [7]:
# prompt: sorry i mean get the parameterss use to train this model like eopch, activation, layers and units. i need to retrieve it from reconstructed_model

# Access the layers of the reconstructed model
for layer in reconstructed_model.layers:
  if isinstance(layer, tf.keras.layers.Dense):
    print("Layer:", layer.name)
    print("Units:", layer.units)
    print("Activation:", layer.activation)
    print("-----")

# Access the optimizer and its parameters (if applicable)
optimizer = reconstructed_model.optimizer
if optimizer:
    print("Optimizer:", optimizer)
    # You might need to access specific attributes based on the optimizer type
    # For example, for Adam:
    if isinstance(optimizer, tf.keras.optimizers.Adam):
        print("Learning Rate:", optimizer.learning_rate.numpy())

Layer: dense_2
Units: 192
Activation: <function relu at 0x7cc0f6933760>
-----
Layer: dense_3
Units: 288
Activation: <function relu at 0x7cc0f6933760>
-----
Layer: dense_4
Units: 192
Activation: <function relu at 0x7cc0f6933760>
-----
Layer: dense_5
Units: 1
Activation: <function linear at 0x7cc0f60fb760>
-----
Optimizer: <keras.src.optimizers.adam.Adam object at 0x7cc0eb36ae30>
Learning Rate: 0.001


In [26]:
# Compile my reconstructed_model
reconstructed_model.compile(optimizer='adam', loss='mean_squared_error')

In [36]:
# use reconstruct model to predict
y_pred = reconstructed_model.predict(input_dict_test)
mae = mean_absolute_error(data_df_test['totalFare'], y_pred)
rmse = root_mean_squared_error(data_df_test['totalFare'], y_pred)
print(f'Mean Absolute Error: {mae}')

39896/39896 [==============================] - 84s 2ms/step
Mean Absolute Error: 324.3405456542969


In [ ]:
# append the predict to the training set
# extract the training predict from the history of 'model'
y_train_predict = model.predict(input_dict)


# append the predict to the training set
data_df['y_train_predict'] = y_train_predict




 34720/127666 [=======>......................] - ETA: 2:37

In [ ]:
# prompt: plot the loss over the epochs

import matplotlib.pyplot as plt

# Assuming you have a history object from your model training
# history = history.history
# Plot the training and validation loss
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.xlim(1,50)
plt.ylim(0,20000)
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:


# Examine the distribution of errors:
errors = data_df_test['totalFare'] - y_pred
plt.hist(errors)
plt.title('Distribution of Errors')
plt.xlabel('Error')
plt.ylabel('Frequency')
plt.show()
# - Ideally, the errors should be normally distributed with a mean of zero.
# - Skewed distributions might suggest problems with your model or data.


# Evaluate the model with different metrics
# from sklearn.metrics import r2_score
# r2 = r2_score(y_test, y_pred)
# print(f"R-squared: {r2}")
# - R-squared measures the goodness of fit.


# Analyze the importance of features:
# - If you have a model that allows you to extract feature importance (e.g., Random Forest), investigate which features are most influential.
# - You might be able to simplify your model by removing less important features.
# - Investigate correlation between features and target variable.

# Consider changing parameters and layers:
# - Overfitting:
#   - Regularization (e.g., L1 or L2 regularization, dropout)
#   - Reduce network complexity (e.g., fewer layers, fewer neurons)
#   - Get more data
# - Underfitting:
#   - Increase network complexity (e.g., more layers, more neurons)
#   - Try a more complex model
# - Slow convergence:
#   - Change learning rate
#   - Change optimizer (e.g., Adam, RMSprop)

# Debugging steps:
# - Check for data leaks between your training and validation sets.
# - Inspect the data for any unusual values, missing values, or inconsistencies.
# - Check your code for potential bugs or errors.
# - Ensure that your model is correctly configured (e.g., input shape, activation functions).
# - Look into activation function and activation rate.
# - Experiment with hyperparameter tuning (e.g., using grid search or random search) to improve the model's performance.

#Example of adding regularization (dropout):
# from tensorflow.keras.layers import Dropout
# x = Dense(128, activation='relu')(x)
# x = Dropout(0.5)(x)  # Add dropout with a rate of 0.5
# x = Dense(64, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(32, activation='relu')(x)
# x = Dense(1)(x)


#Remember that improving a neural network often requires an iterative process of experimentation and refinement.